In [ ]:
import dask.dataframe as dd

In [ ]:
ddf = dd.read_parquet("./data/graf-dataset/")
ddf

In [ ]:
trump_counts = ddf.trump.value_counts().compute()
trump_counts

In [ ]:
trump_counts / trump_counts.sum()

As expected, push is by far the most common. I really can't wrap my head around why the different suits have different frequencies. There's no distinction between colors, just trumps vs not trump so all the suits should be the same number right?!. This unfortunately makes it likely that there is an error somewhere in the dataset generation.

In [ ]:
ddf.fh.value_counts().compute()

At least the number of forehand and rearhand is the same.

In [ ]:
ddf

In [ ]:
ddf[[f"c{i}" for i in range(1, 10)]]

In [ ]:
ddf["c1"].isna().any().compute()

In [ ]:
for i in range(1, 10):
    print(ddf[f"c{i}"].isna().any().compute())

In [ ]:
total_value_counts = []
for i in range(1, 10):
    total_value_counts.append(ddf[f"c{i}"].value_counts().compute())

In [ ]:
import pandas as pd

In [ ]:
total_value_counts

In [ ]:
combined = pd.concat(total_value_counts)
combined = combined.groupby(combined.index).sum()
combined

In [ ]:
assert (combined == 47071640).all(), "Not all cards appear the same number of times."

In [ ]:
total_value_counts = []
for i in range(4):
    total_value_counts.append(ddf[f'ts{i}'].value_counts().compute())

In [ ]:
suit0 = total_value_counts[0]
for values in total_value_counts:
    print((values == suit0).all())

The scores are also all the same with the same frequency, so why the hell are the selected trumps not the same frequency?

Nevermiiind, I figured it out. When there are two trumps with the same score, the first one is picked by argmax so the frequency will be descending.

So, the dataset looks good :) Now we could use this to train a network. Here we can take almost 17 Million examples per trump if we downsample (over 115 mil in total), which certainly is enough. Then for fine-tuning we could take only the very best players and again a balanced random sample of their games.

But play by NN is more important now.